# Predição de Consumo de Combustível

Criação de modelos de aprendizado supervisionado para predizer o consumo de combustível de carros.

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

import preprocessing as pre
import plots
import training as tr

## Pré-processamento

Aqui foi realizado o carregamento dos dados originais de um arquivo _csv_.

In [2]:
original_df = pd.read_csv('data/car_data_original.csv')

Após isso, realizou-se a remoção das instâncias de carros elétricos pois continham muitos atributos faltantes e removeu-se os atributos de consumo em rodovia e cidade pois podem ser altamente colineares com a saída, simplificando demais o trabalho do preditor.

In [3]:
# Removendo os atributos colineares
no_colinear_df = pre.drop_atributes(original_df, ['highway_mpg', 'city_mpg'])

# Removendo as instâncias com NaN
no_nan_df = pre.remove_instances_with_nan(no_colinear_df)

Separou-se os atributos preditores `X` e preditos `y` em seguida.

In [4]:
X = no_nan_df.drop(columns=["combination_mpg"])
y = no_nan_df["combination_mpg"]

Determinou-se o tamanho dos conjuntos de teste e validação.

In [5]:
test_size = 0.1

Após configurar esse parâmetro, colocou-se ao menos uma instância de cada categoria no _dataset_ de treinamento.

Como não há nenhuma garantia que instância pelo menos uma instância de cada tipo estará na etapa de testes, garantiu-se que pelo menos na parte de treinamento os modelos passariam por todos os tipos de modelos de carros, garantindo preditores com mais completude.

In [6]:
train_indices = set()
categorical_features = X.select_dtypes(include=['object', 'bool']).columns
for feature in categorical_features:
    unique_values = X[feature].unique()
    for value in unique_values:
        index = X[X[feature] == value].index[0]
        train_indices.add(index)

train_indices = list(train_indices)
X_train_mandatory = X.loc[train_indices]
y_train_mandatory = y.loc[train_indices]

X = X.drop(train_indices)
y = y.drop(train_indices)

## Divisão de Dados e Treinamento para as Primeiras Avaliações

Separou-se o conjunto de teste do conjunto de treinamento/validação antes de começar qualquer treinamento ou _spot-checking_ de modelos.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

Com essa separação, aplicou-se o treinamento e validação dos diversos modelos com a estratégia de _k-fold cross-validation_.

In [ ]:
methods = [
    "knn",
    "random_forest",
    "linear_regression",
    "neural_networks",
    "svm",
]

# Variáveis de controle para ajustar o número de iterações que calculam o desempenho e aumentam a reprodutibilidade dessa sequência de códigos
n_repeats = 1
k = 9 # k utilizado para o modelo kNN
kfolds = 10
plot_flag = False

for method in methods:
        tr.train(n_repeats, kfolds, k, plot_flag, X_train, y_train, X_train_mandatory, y_train_mandatory, method)

Average MAE using the knn method: 18.19333333333333
Average MSE using the knn method: 113.39333333333333


Average MAE using the random_forest method: 12.446783554973665
Average MSE using the random_forest method: 30.295209939449848


Average MAE using the linear_regression method: 0.7456117217743834
Average MSE using the linear_regression method: 0.930769340281029


Average MAE using the neural_networks method: 1.191887203365788
Average MSE using the neural_networks method: 1.7572170733101142


Average MAE using the svm method: 13.293510177673669
Average MSE using the svm method: 81.27379326677769


